In [8]:
# !pip install wikipedia
# !pip install -U langchain
# !pip install duckduckgo-search
# !pip install httpx==0.24.1

  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/65/f4/dea10c11f0c2ebca298ba57f546940f8944e49c624239183c924ae0e1f81/langchain-0.0.339-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 4.9 MB/s eta 0:00:00a 0:00:010m
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.150
    Uninstalling langchain-0.0.150:
      Successfully uninstalled langchain-0.0.150
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index 0.8.64.post1 requires openai>=1.1.0, but you have openai 0.28.1 which is incompatible.
nemoguardrails 0.5.0 requires fastapi==0.96.0, but you have fastapi 0.92.0 which is incompatible.
nemoguardrails 0.5.0 requires httpx==0.23.3, but you have httpx 0.24.1 which is incompatible.
nemoguardrails 0.5.0 requires langchain==0.0.251, but you have langchain

In [12]:
import yaml, os
from langchain.agents import Tool
from langchain.agents import initialize_agent
from langchain.tools import DuckDuckGoSearchRun
from langchain.chat_models import AzureChatOpenAI
from langchain.utilities import WikipediaAPIWrapper, PythonREPL

In [13]:
with open('cadentials.yaml') as f:
    credentials = yaml.load(f, Loader=yaml.FullLoader)

os.environ["OPENAI_API_TYPE"] = credentials['AD_OPENAI_API_TYPE']
os.environ["OPENAI_API_VERSION"] = credentials['AD_OPENAI_API_VERSION']
os.environ["OPENAI_API_BASE"] = credentials['AD_OPENAI_API_BASE']
os.environ["OPENAI_API_KEY"] = credentials['AD_OPENAI_API_KEY']

### just check some simple tools

In [2]:
wikipedia = WikipediaAPIWrapper()
search = DuckDuckGoSearchRun()
python_repl = PythonREPL()

In [3]:
print(wikipedia.run('Langchain'))

Page: LangChain
Summary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.

Page: Prompt engineering
Summary: Prompt engineering is the process of structuring text that can be interpreted and understood by a generative AI model. A prompt is natural language text describing the task that an AI should perform.A prompt for a text-to-text model can be a query such as "what is Fermat's little theorem?", a command such as "write a poem about leaves falling", a short statement of feedback (for example, "too verbose", "too formal", "rephrase again", "omit this word") or a longer statement including context, instructions, and input data. Prompt engineering may involve phrasing a query, specifying a style, providing relevant context or assi

In [4]:
print(python_repl.run("print(17*2)"))

Python REPL can execute arbitrary code. Use with caution.


34



In [6]:
print(search.run("Tesla stock price?"))

 On Friday 11/17/2023 the closing price of the Tesla share was $234.30 on NAS. Compared to the opening price on Friday 11/17/2023 on NAS of $232.11, this is a gain of 0.93%. Tesla's market... $233.19 −1.27 (0.54%) View Full Chart As of Nov 20, 2023 3:07pm Cboe BZX Real-Time Price Open Unlock Our Analysis with Morningstar Investor Start Free Trial Company Report View Archive Tesla... Start investing The only app that saves & invests your spare change Claim your $20 bonus investment Learn More Via Acorn's Website Tesla and its high-profile CEO Elon Musk have been two of the... Tesla stock has gone up 135% since January, but it's still 40% off its 2021 high. See if this recent strength is building towards a higher stock price for TSLA in 2025. Tesla, Inc. (TSLA) Stock Price, Quote & News - Stock Analysis 241.20 +5.60 (2.38%) At close: Nov 21, 2023, 4:00 PM 240.17 -1.03 (-0.43%) After-hours: Nov 21, 2023, 4:56 PM EST Overview Financials Statistics Forecast Profile Chart 1D 5D 1M YTD 1Y 5Y 

### putting all of them together

In [8]:
python_repl_tool =  Tool(
                        name = "python repl",
                        func=python_repl.run,
                        description="useful for when you need to use python to answer a question. You should input python code"
                        )   

wikipedia_tool = Tool(
                    name='wikipedia',
                    func= wikipedia.run,
                    description="Useful for when you need to look up a topic, country or person on wikipedia"
                    )

duckduckgo_tool = Tool(
                    name='DuckDuckGo Search',
                    func= search.run,
                    description="Useful for when you need to do a search on the internet to find information that another tool can't find. be specific with your input."
                    )

In [9]:
tools = [
        python_repl_tool,
        wikipedia_tool,
        duckduckgo_tool
        ]

In [14]:
llm = AzureChatOpenAI(
                    deployment_name=credentials['AD_DEPLOYMENT_ID'],
                    model_name=credentials['AD_ENGINE'],
                    temperature=0.9, 
                    max_tokens = 256
                    )

In [15]:
zero_shot_agent = initialize_agent(
                                agent="zero-shot-react-description",
                                tools=tools,
                                llm=llm,
                                verbose=True,
                                max_iterations=3,
                                )

In [16]:
print(zero_shot_agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

python repl: useful for when you need to use python to answer a question. You should input python code
wikipedia: Useful for when you need to look up a topic, country or person on wikipedia
DuckDuckGo Search: Useful for when you need to do a search on the internet to find information that another tool can't find. be specific with your input.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [python repl, wikipedia, DuckDuckGo Search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [17]:
zero_shot_agent.run("What is 17*6?")



> Entering new AgentExecutor chain...
I can easily do this calculation in my head, but I can also use python to confirm the answer.
Action: python repl
Action Input: 17*6
Observation: 
Thought:The answer is 102.
Final Answer: 102

> Finished chain.


'102'

In [19]:
zero_shot_agent.run("When was Barak Obama born?")



> Entering new AgentExecutor chain...
I think I can get this information from either Wikipedia or a Python REPL
Action: Wikipedia
Action Input: "Barack Obama"
Observation: Wikipedia is not a valid tool, try one of [python repl, wikipedia, DuckDuckGo Search].
Thought:I'll try DuckDuckGo Search instead
Action: DuckDuckGo Search
Action Input: "Barack Obama birthdate"
Observation: Quick Facts FULL NAME: Barack Hussein Obama II BORN: August 4, 1961 BIRTHPLACE: Honolulu, Hawaii SPOUSE: Michelle Obama (1992-present) Here are 28 of President Obama's biggest accomplishments as President of the United States. 1 - Rescued the country from the Great Recession, cutting the unemployment rate from 10% to 4.7% over ... Gerren Keith Gaynor | Aug 4, 2023 Loading the player... It's been more than six years since former President Barack Obama left office, leaving behind a historic administration that changed the... Only 5 Former U.S. Presidents Are Still Alive — Including the Oldest, Jimmy Carter. Barac

'Barack Obama was born on August 4, 1961.'

In [20]:
zero_shot_agent.run("What is the current price of BTC?")



> Entering new AgentExecutor chain...
I am an AI language model and do not have access to real-time data. I will need to search for an online source that provides the current price of BTC.
Action: DuckDuckGo Search
Action Input: "current price of BTC"
Observation: BTC USD Last update: 5:57 AM, November 21, 2023 Refresh Bitcoin Markets Bitcoin is climbing this week. The current price of Bitcoin is $37,040.40 per BTC. With a circulating supply of 19,548,712 BTC, it means that Bitcoin has a total market cap of $725,127,607,239.44. BITCOIN 37,469.10 +1,988.10 +5.60% XRP RIPPLE 0.64769 +0.01811 +2.88% ETH The live price of Bitcoin is $ 36,080.96 per (BTC / USD) with a current market cap of $ 705.20B USD. 24-hour trading volume is $ 24.33B USD. BTC to USD price is updated in real-time. Bitcoin is -2.01% in the last 24 hours with a circulating supply of 19.54M. BTC Price History USD BTC Price Information 24h Low & High Low: $ 35,545.47 Bitcoin btc Add to Watchlist $36,992.00 1.30% Asset Pri

'The current price of BTC is $37,040.40 per BTC as of 5:57 AM on November 21, 2023 according to my DuckDuckGo Search.'

In [23]:
zero_shot_agent.run("Is 11 a prime number or not?")



> Entering new AgentExecutor chain...
I need to check if 11 is divisible by any integer other than 1 and itself.
Action: python repl
Action Input: 
```python
for i in range(2, 11):
  if 11 % i == 0:
    print("Not prime")
    break
else:
  print("Prime")
```
Observation: SyntaxError('invalid syntax', ('<string>', 1, 1, '```python\n', 1, 2))
Thought:I need to fix the syntax error in the code
Action: python repl
Action Input: 
```python
for i in range(2, 11):
  if 11 % i == 0:
    print("Not prime")
    break
else:
  print("Prime")
```
Observation: SyntaxError('invalid syntax', ('<string>', 1, 1, '```python\n', 1, 2))
Thought:I need to remove the triple backticks from the code before inputting it into the python repl.
Action: python repl
Action Input: 
```python
for i in range(2, 11):
  if 11 % i == 0:
    print("Not prime")
    break
else:
  print("Prime")
```
Observation: SyntaxError('invalid syntax', ('<string>', 1, 1, '```python\n', 1, 2))
Thought:

> Finished chain.


'Agent stopped due to iteration limit or time limit.'